# Calculate Urban Metrics: Shape

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import time

In [3]:
import pandas as pd
import geopandas as gpd
import math

/home/wb411133/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [43]:
# Get reference to GOSTNets
sys.path.append(r"C:\repos\INFRA_SAP")
from infrasap.urban_metrics import *

In [44]:
start_time = time.time()

## To-do
### Calculate for Central Asia
Kazakhstan, Kyrgystan, Tajikistan, Afghanistan, Turkmenistan, and Uzbekistan
### Calculate for ECA (https://www.hurriyetdailynews.com/world-bank-updates-gdp-forecast-for-europe-central-asia-158933)
Afghanistan, Albania, Armenia, Azerbaijan, Belarus, Bosnia and Herzegovina, Bulgaria, Croatia, Georgia, Kazakhstan, Kosovo, Kyrgyzstan, Moldova, Montenegro, North Macedonia, Poland, Romania, Serbia, Tajikistan, Turkey, Turkmenistan, Ukraine, Russia, Hungry, and Uzbekistan.

In [45]:
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\russia_urban_extents_merged_4326.shp"

# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\sample_shps_3.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UBZ_only_FUAs2_geom_fixed.shp"
# shpName = r"C:\repos\GOST_Urban\Notebooks\Implementations\eca_wo_rus_urban_clusters_ghs_pop_smooth_100k_4326_2.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_only_ghs_built_up_extents_4326_geom_fixed_greater_50k.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_ghs_built_up_extents_4326\UZB_only_ghs_built_up_extents_4326_geom_fixed.shp"
# shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\ECA_wo_rus_urban_extents\eca_wo_rus_built_up_extents_4326.shp"
shpName = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_clusters_5k_up_4326.shp"

In [46]:
# import extent
input_shapes_gpd = gpd.read_file(shpName)

In [47]:
# input_shapes_gpd

In [48]:
# proj
# vars(input_shapes_gpd)
input_shapes_gpd._crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [49]:
# make the GeoDataFrame unprojected
input_shapes_gpd = input_shapes_gpd.to_crs("epsg:4326")
input_shapes_gpd._crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [50]:
input_shapes_gpd

,ID,Pop,layer,path,geometry
0,2038,5213.978912,None,None,"POLYGON ((28.77793 55.91414, 28.82094 55.91414..."
1,2355,8086.003265,None,None,"POLYGON ((27.94342 55.81376, 27.95774 55.81376..."
2,2691,12046.157654,None,None,"POLYGON ((27.02823 55.65336, 27.08535 55.65336..."
3,2753,11929.881653,None,None,"POLYGON ((27.60467 55.63333, 27.64749 55.63333..."
4,3060,115285.056396,None,None,"POLYGON ((28.66157 55.57325, 28.67583 55.57325..."
...,...,...,...,...,...
7993,32,6807.330750,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((105.12725 54.81515, 105.18361 54.815..."
7994,28,20279.139618,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((109.32656 55.65336, 109.36940 55.653..."
7995,18,8344.212311,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((107.47868 56.19569, 107.49308 56.195..."
7996,15,5386.852844,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((116.09091 56.33675, 116.10535 56.336..."


In [51]:
start_time = time.time()

In [52]:
# -------------------------------------
# SET UP TEMP WORKSPACE...
TempWS = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\Shape_Metrics_Temp"

# feature grid text file...
gridTxtFile = "%s\\grid.txt" % TempWS

### Loop through each urban extent and calculate its urban metrics. For each urban extent we will reproject it using the UTM zone it belongs in. Then at the end we will unproject it again.

In [53]:
%%time
for index, row in input_shapes_gpd.iterrows():
    print(index)
    metrics = {}

    # creates a temporary GDF for just the row's shape
    temp_gdf = input_shapes_gpd.iloc[[index]]

    # finds its correct UTM zone projection and reprojects it
    temp_gdf_proj = project_gdf(temp_gdf)

    A = temp_gdf_proj.iloc[0].geometry.area
    P = temp_gdf_proj.iloc[0].geometry.length

    # Equal area circle radius...
    r = (
        (temp_gdf_proj.iloc[0].geometry.area / math.pi) ** 0.5
    )  # radius of equal area circle (circle with area equal to shape area) (derived from A = pi * r squared)
    print(f"print r: {r}")
    p = 2 * math.pi * r  # Equal area circle perimeter

    # LIST OF COORDINATES OF FEATURE VERTICES (for single part features)...
    pntLst = []  # stores feature array...
    subVLst = []

    # Step through exterior part of the feature
    for coord in temp_gdf_proj.iloc[0].geometry.exterior.coords:
        # Print the part number
        # print("coord {}:".format(coord))
        # Step through each vertex in the feature
        # Print x,y coordinates of current point
        # print("{}, {}".format(coord[0], coord[1]))
        X, Y = coord[0], coord[1]  # get point XY
        subVLst.append([X, Y])  # add XY to list

    pntLst.append(subVLst)

    # if it has interior polygons
    if len(list(temp_gdf_proj.iloc[0].geometry.interiors)) > 0:
        for poly in list(temp_gdf_proj.iloc[0].geometry.interiors):
            print("new interior polygon")
            subVLst = []
            # Step through each part of the feature
            for coord in poly.coords:
                # print("coord {}:".format(coord))
                # Step through each vertex in the feature
                # Print x,y coordinates of current point
                # print("{}, {}".format(coord[0], coord[1]))
                X, Y = coord[0], coord[1]  # get point XY
                subVLst.append([X, Y])  # add XY to list
            # print(subVLst)
            subVLst.reverse()
            # print(subVLst)
            pntLst.append(subVLst)

    # desired shape area in pixels...
    numPix = 20000

    # calculate pixel size...
    cellsize = (A / numPix) ** 0.5

    # get min and max XY values
    minX, minY, maxX, maxY = (
        temp_gdf_proj.iloc[0].geometry.bounds[0],
        temp_gdf_proj.iloc[0].geometry.bounds[1],
        temp_gdf_proj.iloc[0].geometry.bounds[2],
        temp_gdf_proj.iloc[0].geometry.bounds[3],
    )

    # offset grid by half a pixel...
    minX -= cellsize / 2
    maxY += cellsize / 2

    # centroid coordinates
    centroidXY = (
        temp_gdf_proj.iloc[0].geometry.centroid.x,
        temp_gdf_proj.iloc[0].geometry.centroid.y,
    )
    x_offset, y_offset = 0, 0
    Xc, Yc = centroidXY[0] - x_offset, centroidXY[1] - y_offset

    # generates a list of points within the shape
    featPntLst = generate_featPntLst(
        pntLst, minX, minY, maxX, maxY, cellsize, gridTxtFile
    )

    # NOTE: THE CENTROID IS CURRENTLY USED AS THE CENTER
    # calculate distance of feature points to center...
    D_to_Center, EAC_pix = proximity(featPntLst, Xc, Yc, r)

    # Proximity index (circle / shape)
    # avg distance to center for equal area circle...
    circD = r * (2.0 / 3.0)
    # print(f"print circD: {circD}")
    # print(f"print D_to_Center: {D_to_Center}")
    ProximityIndex = circD / D_to_Center
    metrics["ProximityIndex"] = ProximityIndex

    # Roundness (exchange-index)
    inArea = EAC_pix * cellsize**2
    areaExchange = inArea / A
    metrics["RoundnessIndex"] = areaExchange

    # Cohesion index
    # custom tool calculates approx. average interpoint distances between
    # samples of points in shape...
    shp_interD = interpointDistance(featPntLst)

    # average interpoint distance for equal area circle...
    circ_interD = r * 0.9054

    # cohesion index is ratio of avg interpoint distance of circle to
    # avg interpoint distance of shape...
    CohesionIndex = circ_interD / shp_interD

    metrics["CohesionIndex"] = CohesionIndex

    # Spin index
    # custom tool calculates moment of inertia for shape...
    shpMOI = spin(featPntLst, Xc, Yc)

    # moment of inertia for equal area circle...
    circ_MOI = 0.5 * r**2

    # calculate spin index (circle / shape)...
    Spin = circ_MOI / shpMOI

    metrics["SpinIndex"] = Spin

    # Perimeter index (circle / shape)
    PerimIndex = p / P  # The Perimeter Index
    metrics["PerimIndex"] = PerimIndex

    # Pre-calculations for Depth, Girth, and Dispersion indices

    # print(f"print first 3 of pntLst: {pntLst[0][:3]}")

    # get list of points evenly distributed along perimeter...
    perimPntLst = PerimeterPnts(pntLst, 500)

    # print(f"print first of perimPntLst: {perimPntLst[0]}")

    # ------------------------------------------------------------------------------
    # SECTION 7: CALCULATE DISTANCE OF INTERIOR SHAPE POINTS TO PERIMETER POINTS...

    # custom tool calculates distance of each interior point to nearest perimeter point...
    pt_dToE = pt_distToEdge(featPntLst, perimPntLst)

    # print(f"print max pt_dToE: {pt_dToE[-1]}")

    # Depth index
    # custom tool calculates average distance from interior pixels to nearest edge pixels...
    shp_depth = depth(pt_dToE)

    # depth for equal area circle...
    EAC_depth = r / 3

    # calculate depth index (shape / circle)...
    depthIndex = shp_depth / EAC_depth
    metrics["DepthIndex"] = depthIndex

    # Girth index
    # custom tool calculates shape girth (distance from edge to innermost point)
    # and outputs list position of innermost point...
    shp_Girth = girth(pt_dToE)

    # calculate girth index (shape / circle)...
    girthIndex = shp_Girth / r  # girth of a circle is its radius
    # print(f"print shp_Girth: {shp_Girth}")
    # print(f"print r: {r}")
    metrics["GirthIndex"] = girthIndex

    # Dispersion index
    # custom tool calculates average distance between proximate center and edge points...
    dispersionIndex, avgD = dispersion([Xc, Yc], perimPntLst[0])
    metrics["DispersionIndex"] = dispersionIndex

    # Detour index
    # custom tool creates list of points in the exterior polygon shape
    hullPerim = ConvexHull(pntLst[0])

    # calculate detour index (circle / shape)...
    detourIndex = p / hullPerim
    metrics["DispersionIndex" "DispersionIndex"] = detourIndex

    # Range index
    # custom tool identifies perimeter points that are farthest apart, outputs
    # distance between furthermost points...
    circumCircD = Range(pntLst[0])

    # calculate range index (circle / shape)
    rangeIndex = (2 * r) / circumCircD
    metrics["RangeIndex"] = rangeIndex

    # Put all metrics in a DataFrame
    metrics_scalar = {}
    for k in metrics:
        metrics_scalar[k] = [metrics[k]]
    metrics_df = pd.DataFrame(metrics_scalar)

    # and concatinate it with the row's shape
    new_temp_gdf_proj = pd.concat(
        [temp_gdf_proj.reset_index(drop=True), metrics_df], axis=1
    )

    # print("print new_temp_gdf_proj")
    # print(new_temp_gdf_proj)

    # make it unprojected
    temp_gdf_proj_4326 = new_temp_gdf_proj.to_crs("epsg:4326")

    # put the results of each row into a new DataFrame
    if index == 0:
        # print("creating output_shapes_gpd_4326")
        output_shapes_gpd_4326 = temp_gdf_proj_4326
    else:
        # print(f"print output_shapes_gpd_4326, and index is {index}")
        # print(output_shapes_gpd_4326)
        # print("to append temp_gdf_proj_4326")
        # print(temp_gdf_proj_4326)
        output_shapes_gpd_4326 = output_shapes_gpd_4326.append(
            temp_gdf_proj_4326, ignore_index=True
        )
        # print("output_shapes_gpd_4326 after append")
        # print(output_shapes_gpd_4326)

0
print utm zone: 35
print r: 1383.3588210482546
1
print utm zone: 35
print r: 1694.0441659099426
2
print utm zone: 35
print r: 1785.6210716841833
3
print utm zone: 35
print r: 1597.1201023868887
4
print utm zone: 35
print r: 4225.8664605549575
5
print utm zone: 35
print r: 3830.1659519580217
6
print utm zone: 35
print r: 1873.5068561278742
7
print utm zone: 35
print r: 1693.914926038611
8
print utm zone: 35
print r: 1597.5252991269244
9
print utm zone: 35
print r: 1494.0401836552328
10
print utm zone: 36
print r: 5924.089588981103
11
print utm zone: 35
print r: 2186.8016920181803
12
print utm zone: 35
print r: 2327.993266999866
13
print utm zone: 35
print r: 1694.426299702213
14
print utm zone: 36
print r: 1494.1854396073065
15
print utm zone: 42
print r: 1693.8971241230831
16
print utm zone: 35
print r: 1493.8955269509456
17
print utm zone: 35
print r: 1872.706695895764
18
print utm zone: 35
print r: 2258.69030215062
19
print utm zone: 42
print r: 4689.931872135506
20
print utm zone:

164
print utm zone: 33
print r: 1383.23567475339
165
print utm zone: 36
print r: 1597.0256163428423
166
print utm zone: 33
print r: 1262.5073971252405
167
print utm zone: 36
print r: 1382.7449106253553
168
print utm zone: 34
print r: 1596.7740889492538
169
print utm zone: 35
print r: 2187.0527710556657
170
print utm zone: 34
print r: 2327.78173565375
171
print utm zone: 34
print r: 1383.0028479566859
172
print utm zone: 34
print r: 2187.113660419852
173
print utm zone: 41
print r: 2586.9146125405396
174
print utm zone: 34
print r: 2525.5165847145167
175
print utm zone: 34
print r: 4552.472236013564
176
print utm zone: 34
print r: 1129.0224682818625
177
print utm zone: 33
print r: 1596.6753371699313
178
print utm zone: 33
print r: 1785.5236249566751
179
print utm zone: 34
print r: 1872.2828925413241
180
print utm zone: 33
print r: 1597.1991022727818
181
print utm zone: 36
print r: 1785.2126469828286
182
print utm zone: 34
print r: 1785.759396000151
183
print utm zone: 33
print r: 7338.7

326
print utm zone: 33
print r: 1493.3781082203816
327
print utm zone: 33
print r: 1784.980242227207
328
print utm zone: 41
print r: 1382.7437044264202
329
print utm zone: 33
print r: 1262.7032488199818
330
print utm zone: 34
print r: 2986.9064516892013
331
print utm zone: 34
print r: 1955.5576584819273
332
print utm zone: 34
print r: 2112.409700348648
333
print utm zone: 34
print r: 1693.4338451428932
334
print utm zone: 34
print r: 2257.789642306379
335
print utm zone: 34
print r: 2035.3030906638094
336
print utm zone: 41
print r: 1955.4270520963044
337
print utm zone: 33
print r: 1128.970870309958
338
print utm zone: 33
print r: 1596.9325038078832
339
print utm zone: 34
print r: 1383.2302535152455
340
print utm zone: 36
print r: 2460.750226438016
341
print utm zone: 33
print r: 1955.6488976979615
342
print utm zone: 43
print r: 1693.705163168833
343
print utm zone: 34
print r: 2586.616388915076
344
print utm zone: 33
print r: 3569.8733336538544
345
print utm zone: 36
print r: 3659.4

488
print utm zone: 33
print r: 2821.9956902915155
489
print utm zone: 33
print r: 4337.2338213703615
490
print utm zone: 33
print r: 2524.0779058006033
491
print utm zone: 43
print r: 3869.3533515460076
492
print utm zone: 34
print r: 1596.4312805674165
493
print utm zone: 35
print r: 2524.668040874343
494
print utm zone: 34
print r: 1493.4431893105066
495
print utm zone: 33
print r: 2186.323960922066
496
print utm zone: 34
print r: 2707.573312480431
497
print utm zone: 34
print r: 2460.651124027148
498
print utm zone: 35
print r: 1596.3588085900053
499
print utm zone: 33
print r: 1785.404261085107
500
print utm zone: 34
print r: 3570.542912214782
501
print utm zone: 34
print r: 1955.2315309521687
502
print utm zone: 36
print r: 1784.9188736784586
503
print utm zone: 33
print r: 2034.9960360773787
504
print utm zone: 34
print r: 1693.1983023550074
505
print utm zone: 36
print r: 1596.3432252089783
506
print utm zone: 34
print r: 1596.9413388282776
507
print utm zone: 36
print r: 2524.

650
print utm zone: 33
print r: 1382.379156948543
651
print utm zone: 33
print r: 3910.0798738020667
652
print utm zone: 33
print r: 2034.9325384845938
653
print utm zone: 33
print r: 2327.705045922886
654
print utm zone: 34
print r: 1955.3209204178484
655
print utm zone: 35
print r: 2822.970811116827
656
print utm zone: 36
print r: 2257.4576182649707
657
print utm zone: 33
print r: 1596.2579694917044
658
print utm zone: 36
print r: 1493.122584973123
659
print utm zone: 33
print r: 2986.6321642315893
660
print utm zone: 35
print r: 1955.2935407776358
661
print utm zone: 36
print r: 1871.9310508937272
662
print utm zone: 33
print r: 1382.413077024435
663
print utm zone: 34
print r: 1493.1469747626252
664
print utm zone: 35
print r: 1596.3313337029347
665
print utm zone: 43
print r: 1693.879009670937
666
print utm zone: 34
print r: 1955.1017502827451
667
print utm zone: 42
print r: 1262.3566567427388
668
print utm zone: 34
print r: 1493.0979477420628
669
print utm zone: 34
print r: 2328.

...

7343
print utm zone: 40
print r: 4227.154945674739
7344
print utm zone: 38
print r: 1263.0311671396798
7345
print utm zone: 40
print r: 1694.0216351274828
7346
print utm zone: 39
print r: 2258.803767080229
7347
print utm zone: 39
print r: 1597.1625720666543
7348
print utm zone: 41
print r: 2708.7611065416454
7349
print utm zone: 41
print r: 2767.306373232827
7350
print utm zone: 40
print r: 2113.5388346468226
7351
print utm zone: 39
print r: 2395.7944933434414
7352
print utm zone: 39
print r: 1694.1375509986347
7353
print utm zone: 40
print r: 1873.4382486299914
7354
print utm zone: 40
print r: 1785.803700236878
7355
print utm zone: 38
print r: 1494.5954696568901
7356
print utm zone: 40
print r: 1263.1303361595585
7357
print utm zone: 39
print r: 1694.633940823362
7358
print utm zone: 39
print r: 2648.612700268524
7359
print utm zone: 39
print r: 1693.8021019296218
7360
print utm zone: 39
print r: 2035.9041576495238
7361
print utm zone: 39
print r: 4337.1656674769865
7362
print utm zon

7502
print utm zone: 39
print r: 3658.525421588643
7503
print utm zone: 38
print r: 2259.915981626533
7504
print utm zone: 40
print r: 2933.2825418491516
7505
print utm zone: 38
print r: 1597.9451404340457
7506
print utm zone: 39
print r: 1493.441167205007
7507
print utm zone: 38
print r: 1873.377142717754
7508
print utm zone: 39
print r: 1694.0522315677372
7509
print utm zone: 39
print r: 2709.558615335842
7510
print utm zone: 39
print r: 2112.356615482792
7511
print utm zone: 38
print r: 1494.599279781974
7512
print utm zone: 38
print r: 1493.5718316925193
7513
print utm zone: 37
print r: 2766.8366217738744
7514
print utm zone: 39
print r: 1785.1534718902583
7515
print utm zone: 37
print r: 8749.656305221992
7516
print utm zone: 39
print r: 1597.228284173147
7517
print utm zone: 36
print r: 3616.6818181956532
7518
print utm zone: 39
print r: 1382.9241514405933
7519
print utm zone: 38
print r: 2036.6155245843668
7520
print utm zone: 40
print r: 1597.0403589090033
7521
print utm zone: 

7661
print utm zone: 38
print r: 1956.458957227283
7662
print utm zone: 39
print r: 1596.4390404677079
7663
print utm zone: 37
print r: 1597.6785808020607
7664
print utm zone: 40
print r: 1956.064322546806
7665
print utm zone: 37
print r: 1873.1192876768794
7666
print utm zone: 39
print r: 1693.2745217653141
7667
print utm zone: 38
print r: 2113.4416440106183
7668
print utm zone: 38
print r: 1597.2194635522617
7669
print utm zone: 37
print r: 1786.077506475456
7670
print utm zone: 40
print r: 1596.493016947041
7671
print utm zone: 37
print r: 1956.8417493531317
7672
print utm zone: 40
print r: 1693.3219468315428
7673
print utm zone: 37
print r: 2766.9074524385132
7674
print utm zone: 39
print r: 1596.819016954043
7675
print utm zone: 38
print r: 1263.2484313511327
7676
print utm zone: 38
print r: 1693.5048538494282
7677
print utm zone: 38
print r: 1597.6622982127822
7678
print utm zone: 39
print r: 1493.3736012794009
7679
print utm zone: 37
print r: 6894.659586181376
7680
print utm zon

7820
print utm zone: 49
print r: 2707.9406763218913
7821
print utm zone: 49
print r: 2035.8605384082218
7822
print utm zone: 49
print r: 5785.61425788105
7823
print utm zone: 49
print r: 1493.737804651893
7824
print utm zone: 49
print r: 2112.7136909662418
7825
print utm zone: 50
print r: 2257.6819254800657
7826
print utm zone: 49
print r: 1955.8288108628567
7827
print utm zone: 50
print r: 1382.6061961833852
7828
print utm zone: 50
print r: 1872.178213689754
7829
print utm zone: 50
print r: 1382.5487876529492
7830
print utm zone: 50
print r: 1693.500430546692
7831
print utm zone: 38
print r: 1871.9650379768698
7832
print utm zone: 37
print r: 1692.7808314669196
7833
print utm zone: 50
print r: 2394.522760344537
7834
print utm zone: 50
print r: 1693.196449256385
7835
print utm zone: 50
print r: 1872.078237548657
7836
print utm zone: 50
print r: 2035.5378294809739
7837
print utm zone: 37
print r: 2647.690628069464
7838
print utm zone: 49
print r: 1693.9734103444232
7839
print utm zone: 

7979
print utm zone: 48
print r: 1872.1494362606131
7980
print utm zone: 48
print r: 1596.5834398985012
7981
print utm zone: 48
print r: 6079.780457575745
7982
print utm zone: 48
print r: 1955.5822639918877
7983
print utm zone: 48
print r: 1596.6979165960565
7984
print utm zone: 49
print r: 1596.5617383852768
7985
print utm zone: 48
print r: 3744.2085800474297
7986
print utm zone: 53
print r: 1695.2334354568484
7987
print utm zone: 52
print r: 1495.0351205242118
7988
print utm zone: 52
print r: 1384.1304841151748
7989
print utm zone: 50
print r: 1873.7619366146323
7990
print utm zone: 49
print r: 1129.909323292384
7991
print utm zone: 52
print r: 1129.9701178261846
7992
print utm zone: 51
print r: 1787.0217007758624
7993
print utm zone: 48
print r: 1785.3345371333971
7994
print utm zone: 49
print r: 1956.27882277047
7995
print utm zone: 48
print r: 1694.4848459687087
7996
print utm zone: 50
print r: 978.1283807111066
7997
print utm zone: 52
print r: 1495.0562779744319
Wall time: 2d 9h 

In [54]:
output_shapes_gpd_4326

,ID,Pop,layer,path,geometry,ProximityIndex,RoundnessIndex,CohesionIndex,SpinIndex,PerimIndex,DepthIndex,GirthIndex,DispersionIndex,DispersionIndexDispersionIndex,RangeIndex
0,2038,5213.978912,None,None,"POLYGON ((28.77793 55.91414, 28.82094 55.91414...",0.810037,0.67250,0.806304,0.603130,0.639588,0.553369,0.451982,0.679351,0.664965,0.616637
1,2355,8086.003265,None,None,"POLYGON ((27.94342 55.81376, 27.95774 55.81376...",0.729066,0.60920,0.723960,0.457315,0.611062,0.561151,0.513182,0.654913,0.638330,0.489989
2,2691,12046.157654,None,None,"POLYGON ((27.02823 55.65336, 27.08535 55.65336...",0.884751,0.77265,0.880896,0.757344,0.634816,0.588428,0.619353,0.769597,0.721842,0.719950
3,2753,11929.881653,None,None,"POLYGON ((27.60467 55.63333, 27.64749 55.63333...",0.951826,0.84640,0.944838,0.884533,0.712323,0.698080,0.697881,0.850222,0.738788,0.791814
4,3060,115285.056396,None,None,"POLYGON ((28.66157 55.57325, 28.67583 55.57325...",0.698634,0.54810,0.703135,0.468496,0.327686,0.324688,0.415226,0.672839,0.623278,0.587053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7993,32,6807.330750,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((105.12725 54.81515, 105.18361 54.815...",0.807690,0.65675,0.796686,0.594423,0.513515,0.488842,0.518141,0.688186,0.686368,0.542625
7994,28,20279.139618,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((109.32656 55.65336, 109.36940 55.653...",0.922379,0.81360,0.916525,0.811018,0.568772,0.592381,0.621923,0.797682,0.734598,0.619863
7995,18,8344.212311,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((107.47868 56.19569, 107.49308 56.195...",0.785950,0.64830,0.782908,0.545774,0.591756,0.601559,0.561962,0.697812,0.663385,0.486118
7996,15,5386.852844,27_urban_clusters_ghs_pop_russia,C:/repos/GOST_Urban/Notebooks/Implementations/...,"POLYGON ((116.09091 56.33675, 116.10535 56.336...",0.804477,0.68570,0.793309,0.574706,0.605181,0.586779,0.621907,0.677148,0.595039,0.490075


In [55]:
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_metrics_results_russia"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\UZB_only_GHS_urban_extents_results_all"
# output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\eca_urban_metrics_results_wo_rus_all"
output = r"C:\Users\war-machine\Documents\world_bank_work\UZB_project\metrics_shape_tool\all_urban_extents_results_5k_up"

In [56]:
# save as CSV
# output_shapes_gpd_4326.to_csv(output + r"\ECA_russia_urban_metrics_100k_shape.csv")
# output_shapes_gpd_4326.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_shape.csv")
# output_shapes_gpd_4326.to_csv(output + r"\UZB_only_urban_metrics_urban_extents_all_shape.csv")
output_shapes_gpd_4326.to_csv(output + r"\all_urban_metrics_5k_up_shape.csv")

In [57]:
print(f"total time to process: {time.time()-start_time}")

total time to process: 205300.34797525406
